In [1]:
%cd cache-prefetchers
! pip install sympy
from IPython import get_ipython

def git_pull():
    !git pull > /dev/null

get_ipython().events.register('pre_run_cell', git_pull)


/notebooks/cache-prefetchers


In [38]:
%load_ext autoreload
%autoreload 3


import matplotlib.pyplot as plt
import numpy as np
from typing import List, Tuple, Set
import abc
import torch
import dataclasses
import pandas as pd
import os
import copy
import time
import tqdm
from opt import opt_cache_simulation


from cache import Cache
from eviction import LRU, DOM, MQ, RandomEvictor
from prefetch import Next, Markov, EnsamblePrefetcher
from prob_model import FFM
from markov_generator import RandomMarkovGenerator
from fieldfm import train_ffm, FieldAwareFactorizationMachine, FieldAwareFactorizationMachineModel


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
@dataclasses.dataclass
class TestResult:

    cache_name: str
    miss_count: int
    trace_length: int
    unique_addresses: int
    execution_time: float


    def __repr__(self) -> str:
        return (
            f"Cache name: {self.cache_name}\n"
            f"Miss count: {self.miss_count}\n"
            f"Trace length: {self.trace_length}\n"
            f"Unique addresses: {self.unique_addresses}\n"
            f"Miss rate: {self.miss_count / self.trace_length}\n"
            f"Execution time: {self.execution_time}"
        )



def test_cache(cache: Cache, requests: List[int]) -> TestResult:
    """Returns cache miss count for given cache and requests."""
    cache = copy.deepcopy(cache)
    time_start = time.time()

    misses = 0
    for r in tqdm.tqdm(requests):
        if not cache.read(r):
            misses += 1

    time_end = time.time()

    return TestResult(
        cache_name=cache.name,
        miss_count=misses,
        trace_length=len(requests),
        unique_addresses=len(set(requests)),
        execution_time=time_end - time_start,
    )


def get_opt_for_sequence(sequence: list[int], cache_size=8) -> TestResult:
    total_length = 0
    unique_addresses = set()

    unique_addresses.update(sequence)

    misses = opt_cache_simulation(sequence, cache_size)

    return TestResult(
        cache_name="OPT",
        miss_count=misses,
        unique_addresses=len(set(unique_addresses)),
        trace_length=total_length,
        execution_time=None
    )

def test_markov_sequences(caches: List[Cache], sequences: List[List[int]], cache_size: int):
    file_name = f"markov_sequences_{cache_size}.csv"

    if os.path.exists(file_name):
        res = pd.read_csv(file_name, index_col=0)
    else:
        res = pd.DataFrame()

    for i, seq in enumerate(sequences):
        if not (len(res) > 0 and len(res[(res['cache'] == 'OPT') & (res['seq'] == i)]) > 0):
            opt_res = get_opt_for_sequence(seq, cache_size)
            new_row = pd.DataFrame([{
                'seq': i,
                'cache': 'OPT',
                'miss_count': opt_res.miss_count,
                'trace_length': opt_res.trace_length,
                'unique_addresses': opt_res.unique_addresses,
                'execution_time': opt_res.execution_time
            }])
            res = pd.concat([res, new_row])
            res.to_csv(file_name)

        for cache in caches:
            if len(res) > 0 and len(res[(res['cache'] == cache.name) & (res['seq'] == i)]) > 0:
                continue

            print("=======================================")
            print(f"Sequence {i}, cache: {cache.name}")
            print("=======================================")
            try:
                test_result = test_cache(cache, seq)
            except Exception as exc:
                print("ERROR", exc)
                continue

            new_row = pd.DataFrame([{
                'seq': i,
                'cache': cache.name,
                'miss_count': test_result.miss_count,
                'trace_length': test_result.trace_length,
                'unique_addresses': test_result.unique_addresses,
                'execution_time': test_result.execution_time
            }])
            res = pd.concat([res, new_row])
            res.to_csv(file_name)

    res['miss_rate'] = res['miss_count'] / res['trace_length']
    res = res.sort_values(by=['seq', 'miss_rate'])

    return res


In [90]:
SEQ_LEN = 50000
N = 500  # Address space size
SEQ_COUNT = 3  # Number of sequences to generate
CACHE_SIZE = 16

sequences = [
    RandomMarkovGenerator(N).generate_sequence(SEQ_LEN)
    for _ in range(SEQ_COUNT)
]


In [130]:
import random
from distr import test_on_sequence

misses, losses = test_on_sequence(sequences[0], CACHE_SIZE, N, greedy=False, history_size=2)

print(misses)
plt.plot(losses)


  2%|▏         | 1100/50000 [00:02<01:32, 528.76it/s]


KeyboardInterrupt: 

In [94]:
random_cache = Cache(
    name="RandomEvictor",
    eviction_strategy=RandomEvictor(),
    size=CACHE_SIZE
)
random_cache = copy.deepcopy(random_cache)
test_res = test_cache(random_cache, sequences[0])
print(test_res)


100%|██████████| 50000/50000 [00:00<00:00, 88006.03it/s]

Cache name: RandomEvictor
Miss count: 48386
Trace length: 50000
Unique addresses: 500
Miss rate: 0.96772
Execution time: 0.569922924041748


In [24]:
caches = [
    Cache(
        name="RandomEvictor",
        eviction_strategy=RandomEvictor(),
        size=CACHE_SIZE
    ),
    Cache(
        name="LRU",
        eviction_strategy=LRU(),
        size=CACHE_SIZE
    ),
    Cache(
        name="MQ",
        eviction_strategy=MQ(),
        size=CACHE_SIZE
    ),
    Cache(
        name="MQ + MARKOV(3)",
        eviction_strategy=MQ(),
        prefetch_strategy=Markov(3),
        size=CACHE_SIZE
    ),
    Cache(
        name="DOM(FFM(n=N, h=1, k=10))",
        eviction_strategy=DOM(FFM(n=N, h=1, k=10, my_ffm=False), train_interval=50),
        size=CACHE_SIZE
    ),
    Cache(
        name="RandomEvictor + NEXT",
        eviction_strategy=RandomEvictor(),
        prefetch_strategy=Next(),
        size=CACHE_SIZE
    ),
    Cache(
        name="LRU + NEXT",
        eviction_strategy=LRU(),
        prefetch_strategy=Next(),
        size=CACHE_SIZE
    ),
    Cache(
        name="MQ + NEXT",
        eviction_strategy=MQ(),
        prefetch_strategy=Next(),
        size=CACHE_SIZE
    ),
    Cache(
        name="DOM(FFM(n=N, h=14, k=10)) + NEXT",
        eviction_strategy=DOM(FFM(n=N, h=14, k=10), train_interval=50),
        prefetch_strategy=Next(),
        size=CACHE_SIZE
    ),
    Cache(
        name="RandomEvictor + MARKOV(3)",
        eviction_strategy=RandomEvictor(),
        prefetch_strategy=Markov(3),
        size=CACHE_SIZE
    ),
    Cache(
        name="LRU + MARKOV(3)",
        eviction_strategy=LRU(),
        prefetch_strategy=Markov(3),
        size=CACHE_SIZE
    ),
    Cache(
        name="DOM(FFM(n=N, h=14, k=10)) + MARKOV(3)",
        eviction_strategy=DOM(FFM(n=N, h=14, k=10)),
        prefetch_strategy=Markov(3),
        size=CACHE_SIZE
    ),
][:4]


import itertools
H = [1]
K = [10]
TRAIN_INTERVAL = [50]
SAMPLES = [400]
EPOCHS = [2]
WD = [0.1]
LR = [0.001, 0.01]
WITH_NEUTRAL = [True, False]


grid = itertools.product(
    H, K, TRAIN_INTERVAL, SAMPLES, EPOCHS, WD, LR, WITH_NEUTRAL
)

for h, k, train_interval, samples, epochs, wd, lr, with_neutral in grid:
    name = (
        f"DOM(FFM("
        f"n=N "
        f"h={h} "
        f"k={k} "
        f"e_s={samples} "
        f"t_i={train_interval} "
        f"e={epochs} "
        f"lr={lr} "
        f"wd={wd} "
        f"neutral={with_neutral}"
        "))"
    )

    cache = Cache(
        name=name,
        eviction_strategy=DOM(
            FFM(
                n=N, h=h, k=k, epoch_samples=samples, lr=lr, wd=wd, epochs=epochs,
                with_neutral=with_neutral
            ),
            train_interval=train_interval
        ),
        size=CACHE_SIZE
    )
    caches.append(cache)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
test_markov_sequences(caches, sequences, CACHE_SIZE)

Sequence 1, cache: DOM(FFM(n=N h=1 k=10 e_s=400 t_i=50 e=2 lr=0.001 wd=0.1 neutral=False))


100%|██████████| 50000/50000 [05:13<00:00, 159.61it/s]


Sequence 1, cache: DOM(FFM(n=N h=1 k=10 e_s=400 t_i=50 e=2 lr=0.01 wd=0.1 neutral=True))


100%|██████████| 50000/50000 [05:13<00:00, 159.54it/s]


Sequence 1, cache: DOM(FFM(n=N h=1 k=10 e_s=400 t_i=50 e=2 lr=0.01 wd=0.1 neutral=False))


 18%|█▊        | 8970/50000 [00:59<04:30, 151.69it/s]


KeyboardInterrupt: 

# Tests on HUAWEI traces

In [122]:
from hwi_dataset import read_trace, get_test_set, get_reads, get_page_requests
from collections import defaultdict
test_set = get_test_set()

values = defaultdict(int)

times = []
all_length = 0
test_dir = '3'
all_unique = set()
all_unique_pages = 0
all_pages_length = 0

for v, volume in enumerate(test_set[test_dir]):
    print("volume", v)

    times_volume = []
    unique_volume = set()
    length_volume = 0
    unique_pages_volume = set()
    length_pages_volume = 0

    for p, path in enumerate(volume):
        print("path", path)
        trace = read_trace(test_set[test_dir][v][p])
        reads = trace[trace['tp'] == 'READ']
        print(reads['volumeId'].unique())

        times_volume.extend(list(reads['last time nano'].values))
        unique_volume.update(reads['objLba'].unique())

        length_volume += len(reads)

        print('length path', len(reads))

        pages = get_page_requests(get_reads(read_trace(path)), page_size=16*1024)
        unique_pages_path = len(set(pages))
        unique_pages_volume.update(set(pages))
        length_pages_volume += len(pages)
        print('unique pages within path', unique_pages_path)


        print("=======================================\n")

    print('is times sorted within volume?' , all(times_volume[i] <= times_volume[i+1] for i in range(len(times_volume)-1)))
    print('unique addresses within volume', len(unique_volume))
    print('length volume', length_volume)
    print('unique pages within volume', len(unique_pages_volume))

    times.extend(times_volume)
    all_unique.update(unique_volume)
    all_length += length_volume
    all_unique_pages += len(unique_pages_volume)
    all_pages_length += length_pages_volume
    print("##################\n")

print('is times sorted?' , all(times[i] <= times[i+1] for i in range(len(times)-1)))
print('unique addresses', len(all_unique))
print("all length", all_length)
print("all unique pages", all_unique_pages)
print("all pages length", all_pages_length)
trace = read_trace(test_set[test_dir][0][0])
trace


volume 0
path ../trace/Trace/3/trace_150_0/cache_trace_150_0_6
['0x96']
length path 2599
unique pages within path 211

path ../trace/Trace/3/trace_150_0/cache_trace_150_0_7
['0x96']
length path 2333
unique pages within path 191

path ../trace/Trace/3/trace_150_0/cache_trace_150_0_16
['0x96']
length path 2410
unique pages within path 206

path ../trace/Trace/3/trace_150_0/cache_trace_150_0_17
['0x96']
length path 1401
unique pages within path 98

path ../trace/Trace/3/trace_150_0/cache_trace_150_0_19
['0x96']
length path 2760
unique pages within path 170

path ../trace/Trace/3/trace_150_0/cache_trace_150_0_20
['0x96']
length path 1955
unique pages within path 135

path ../trace/Trace/3/trace_150_0/cache_trace_150_0_22
['0x96']
length path 2749
unique pages within path 153

path ../trace/Trace/3/trace_150_0/cache_trace_150_0_23
['0x96']
length path 237
unique pages within path 24

path ../trace/Trace/3/trace_150_0/cache_trace_150_0_24
['0x96']
length path 2701
unique pages within path 17

,timestamp,tp,opCode,result,volumeId,objId,objLba,length,timepoint,last time nano
0,Fri Apr 7 01:08:45 2017,READ,0x128001,0,0x96,0x96,155774464,256,8,340745845685067
1,Fri Apr 7 01:08:45 2017,READ_DONE,0x128001,0,0x96,0x96,155774464,256,8,340745845748128
2,Fri Apr 7 01:08:45 2017,READ,0x128001,0,0x96,0x96,155774720,256,8,340745847651115
3,Fri Apr 7 01:08:45 2017,READ_DONE,0x128001,0,0x96,0x96,155774720,256,8,340745847669217
4,Fri Apr 7 01:08:45 2017,DESTAGE,0x13218000,0,0x96,0x96,173140992,512,8,340745866793606
...,...,...,...,...,...,...,...,...,...,...
11195,Fri Apr 7 01:09:42 2017,WRITE_DONE,0x13218000,0,0x96,0x96,225095680,496,8,340802706843319
11196,Fri Apr 7 01:09:42 2017,WRITE,0x13218000,0,0x96,0x96,225096672,496,8,340802707018769
11197,Fri Apr 7 01:09:42 2017,WRITE_DONE,0x13218000,0,0x96,0x96,225096176,496,8,340802707080023
11198,Fri Apr 7 01:09:42 2017,WRITE,0x13218000,0,0x96,0x96,225097168,496,8,340802707320931


In [121]:
page_size = 16*1024
test_dir_name = '3'
test_set = get_test_set()
requests = []

for volume_paths in sorted(test_set[test_dir_name])[:1]:
    print(f"Volume {volume_paths[0].split('/')[-2]}")

    for path in tqdm.tqdm(volume_paths):
        requests.extend(get_page_requests(get_reads(read_trace(path)), page_size=page_size))

print("unique pages", len(set(requests)))
print("length", len(requests))

Volume trace_150_0


100%|██████████| 24/24 [00:00<00:00, 31.21it/s]

unique pages 2577
length 47855


In [118]:
# bce loss
criterion = torch.nn.BCELoss()

criterion(torch.Tensor([1]).view(-1, 1), torch.Tensor([.85]).view(-1, 1))

tensor(15.0000)

In [123]:
from distr_dom import test_on_sequence as test_on_sequence_dom
misses, misses_history, losses, evict_counts = test_on_sequence_dom(
    requests, cache_size=16, greedy=False, history_size=14, n_pages=2600, hidden_size=128,
    n_deltas=128, emb_size=32, delta_emb_size=0, lr=1e-3, train_interval=100,
    batch_size=128, train_samples=1000, train_next_delta=False, hash_mapping=False,
)

print(misses)

Misses: 3 Loss p(a,b): 0 Loss delta: 0:   0%|          | 177/47855 [00:00<00:26, 1768.48it/s]/Users/robertlaskowski/Desktop/studia/projects/prefetching/IDEAS_prefetchers/distr_dom.py:273: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  prob_target = torch.tensor(batch.pop("prob")).float()


Misses: 181 Loss p(a,b): 13.051169514656067 Loss delta: 0:   4%|▎         | 1690/47855 [00:13<06:04, 126.80it/s]


KeyboardInterrupt: 

In [68]:
def test_random_evictor(requests: List[int], cache_size: int) -> TestResult:
    cache = set()
    misses = 0
    for r in tqdm.tqdm(requests):
        if len(cache) < cache_size:
            cache.add(r)
        elif r not in cache:
            evict_el = random.choice(list(cache))
            cache.remove(evict_el)
            misses += 1

            cache.add(r)
    print(f"Misses: {misses}")

    cache = Cache(
        name="RandomEvictor",
        eviction_strategy=RandomEvictor(),
        size=cache_size
    )
    misses2 = 0
    for r in tqdm.tqdm(requests):
        misses2 += int(not cache.read(r))
    print(f"Misses2: {misses2}")
    cache = set()
    misses3 = 0
    for r in tqdm.tqdm(requests):
        if len(cache) < cache_size:
            cache.add(r)
        elif r not in cache:
            evict_el = list(cache)[0]
            cache.remove(evict_el)
            misses3 += 1

    print(f"Misses3: {misses3}")


    return misses

test_random_evictor(requests, 16)

100%|██████████| 112913/112913 [00:00<00:00, 2288699.57it/s]


Misses: 6703


100%|██████████| 112913/112913 [00:03<00:00, 34817.21it/s]


Misses2: 6694


100%|██████████| 112913/112913 [00:00<00:00, 2401396.69it/s]

Misses3: 9403


6703

In [43]:

def get_opt_for_test_dir(test_set_dir, page_size=16*1024, cache_size=8) -> TestResult:
    requests = []
    for volume_paths in sorted(test_set[test_set_dir]):
        for path in volume_paths:
            requests.extend(get_page_requests(get_reads(read_trace(path)), page_size=page_size))


    misses = opt_cache_simulation(requests, cache_size)

    return TestResult(
        cache_name="OPT",
        miss_count=misses,
        unique_addresses=len(set(requests)),
        trace_length=len(requests),
        execution_time=None
    )


In [84]:
import pandas as pd
import os

test_dirs = [
    #'1',
    '2',
    #'3',
    '11',
    #'scenario_test_trace_simple/VDI_virus_scan'
]

def test_case(test_dir_name: str, cache: Cache, page_size=16*1024) -> TestResult:
    start_time = time.time()

    cache_factory = lambda: copy.deepcopy(cache)
    cache = cache_factory()
    requests = []
    total_misses = 0
    for i, volume_paths in enumerate(sorted(test_set[test_dir_name])):
        if test_dir_name == '11':
            if i > 0:
                break

        for path in volume_paths:
            requests.extend(get_page_requests(get_reads(read_trace(path)), page_size=page_size))

    for r in tqdm.tqdm(requests):
        hit = cache.read(r)
        total_misses += int(not hit)


    return TestResult(
        cache_name=cache.name,
        miss_count=total_misses,
        unique_addresses=len(set(requests)),
        trace_length=len(requests),
        execution_time=round(time.time() - start_time, 3)
    )

def test(caches: List[Cache], test_dirs: List[str], cache_size, page_size=16*1024) -> pd.DataFrame:
    file_name = f'results_{page_size}_{cache_size}.csv'
    if os.path.exists(file_name):
        res = pd.read_csv(file_name, dtype={'test_dir': str}, index_col=0)
    else:
        res = pd.DataFrame()

    for test_dir in test_dirs:
        for cache in caches:

            if len(res) > 0 and len(res[(res['cache'] == cache.name) & (res['test_dir'] == test_dir)]) > 0:
                continue


            print("=======================================")
            print(f"Test dir: {test_dir}, cache: {cache.name}")
            print("=======================================")
            try:
                test_result = test_case(test_dir, cache, page_size=page_size)
            except Exception as exc:
                print("ERROR", exc)
                continue

            new_row = pd.DataFrame([{
                'test_dir': test_dir,
                'cache': cache.name,
                'miss_count': test_result.miss_count,
                'trace_length': test_result.trace_length,
                'unique_addresses': test_result.unique_addresses,
                'page_size': page_size,
                'execution_time': test_result.execution_time
            }])
            res = pd.concat([res, new_row])
            res.to_csv(file_name)

        if len(res) > 0 and len(res[(res['cache'] == 'OPT') & (res['test_dir'] == test_dir)]) > 0:
            continue

        opt_res = get_opt_for_test_dir(test_dir, page_size=page_size, cache_size=cache_size)
        new_row = pd.DataFrame([{
            'test_dir': test_dir,
            'cache': 'OPT',
            'miss_count': opt_res.miss_count,
            'trace_length': opt_res.trace_length,
            'unique_addresses': opt_res.unique_addresses,
            'page_size': page_size,
            'execution_time': opt_res.execution_time
        }])
        res = pd.concat([res, new_row])
        res.to_csv(file_name)

    res['miss_rate'] = res['miss_count'] / res['trace_length']
    res = res.sort_values(by=['test_dir', 'miss_rate'])

    return res


In [46]:

caches_size_16 = [
    Cache(
        "RandomEvictor",
        eviction_strategy=RandomEvictor(),
        size=16
    ),
    Cache(
        "LRU",
        eviction_strategy=LRU(),
        size=16
    ),
    Cache(
        "MQ",
        eviction_strategy=MQ(max_frequency=8, life_time=10, qout_size=8*4),
        size=16
    ),
    Cache(
        "DOM(FFM(n=10000,k=10,h=14))",
        eviction_strategy=DOM(prob_model=FFM(n=10000, k=10, h=14)),
        size=16
    ),
    Cache(
        "LRU + Markov(3)",
        eviction_strategy=LRU(),
        prefetch_strategy=Markov(3),
        size=16
    ),
    Cache(
        "LRU + NEXT",
        eviction_strategy=LRU(),
        prefetch_strategy=Next(),
        size=16
    ),
    Cache(
        "MQ + NEXT",
        eviction_strategy=MQ(max_frequency=8, life_time=10, qout_size=8*4),
        prefetch_strategy=Next(),
        size=16
    ),
    Cache(
        "MQ + Markov(3)",
        eviction_strategy=MQ(max_frequency=8, life_time=10, qout_size=8*4),
        prefetch_strategy=Markov(3),
        size=16
    ),
    Cache(
        "DOM(FFM(n=10000,k=10,h=14)) + NEXT",
        eviction_strategy=DOM(prob_model=FFM(n=1000, k=10, h=14)),
        prefetch_strategy=Next(),
        size=16
    ),
][:3]


In [85]:
test(caches_size_16, test_dirs, 16, page_size=16*1024)


Test dir: 2, cache: RandomEvictor


100%|██████████| 112913/112913 [00:02<00:00, 51489.74it/s]


Test dir: 2, cache: LRU


100%|██████████| 112913/112913 [00:02<00:00, 54287.13it/s]


Test dir: 2, cache: MQ


...

Test dir: 11, cache: LRU


100%|██████████| 18846/18846 [00:00<00:00, 54586.70it/s]


Test dir: 11, cache: MQ


100%|██████████| 18846/18846 [00:00<00:00, 32509.52it/s]
/var/folders/wv/wmn__gfj1j188w6fb4gdzvn80000gn/T/ipykernel_53921/108469552.py:88: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  res = pd.concat([res, new_row])


,test_dir,cache,miss_count,trace_length,unique_addresses,page_size,execution_time,miss_rate
0,11,OPT,40009,57867,12199,16384,NaN,0.691396
0,11,RandomEvictor,14666,18846,5001,16384,0.653,0.778202
0,11,LRU,14690,18846,5001,16384,0.613,0.779476
0,11,MQ,14696,18846,5001,16384,0.844,0.779794
0,2,OPT,5557,112913,2797,16384,NaN,0.049215
0,2,MQ,6359,112913,2797,16384,4.899,0.056318
0,2,LRU,6410,112913,2797,16384,3.602,0.056769
0,2,RandomEvictor,6679,112913,2797,16384,3.790,0.059152


In [533]:

caches_size_64 = [
    Cache(
        "RandomEvictor",
        eviction_strategy=RandomEvictor(),
        size=64
    ),
    Cache(
        "LRU",
        eviction_strategy=LRU(),
        size=64
    ),
    Cache(
        "MQ",
        eviction_strategy=MQ(max_frequency=8, life_time=10, qout_size=8*4),
        size=64
    ),
    Cache(
        "LRU + NEXT",
        eviction_strategy=LRU(),
        prefetch_strategy=Next(),
        size=64
    ),
    Cache(
        "LRU + NEXT",
        eviction_strategy=LRU(),
        prefetch_strategy=Next(),
        size=64
    ),
    Cache(
        "LRU + Markov(3)",
        eviction_strategy=LRU(),
        prefetch_strategy=Markov(3),
        size=16
    ),
    Cache(
        "MQ + NEXT",
        eviction_strategy=MQ(max_frequency=8, life_time=10, qout_size=8*4),
        prefetch_strategy=Next(),
        size=64
    ),
    Cache(
        "MQ + Markov(3)",
        eviction_strategy=MQ(max_frequency=8, life_time=10, qout_size=8*4),
        prefetch_strategy=Markov(3),
        size=64
    ),
]
test(caches_size_64, test_dirs, 64, page_size=16*1024)


,test_dir,cache,miss_count,trace_length,unique_addresses,page_size,execution_time,miss_rate
0,1,OPT,961,61932,14281,16384,None,0.015517
0,1,MQ,48394,61932,14281,16384,9.215,0.781405
0,1,LRU,48415,61932,14281,16384,3.677,0.781744
0,1,RandomEvictor,48952,61932,14281,16384,3.942,0.790415
0,1,MQ + Markov(3),49298,61932,14281,16384,8.863,0.796002
0,1,MQ + NEXT,49328,61932,14281,16384,9.126,0.796486
0,1,LRU + NEXT,49363,61932,14281,16384,4.099,0.797052
0,1,LRU + Markov(3),51446,61932,14281,16384,4.332,0.830685
0,1,"DOM(FFM(n=10000,k=10,h=6))",52467,61932,14281,16384,2194.868,0.847171
0,1,"DOM(FFM(n=10000,k=10,h=6)) + NEXT",57120,61932,14281,16384,4587.174,0.922302


## Tests on smaller page sizes

In [ ]:

caches_size_16_smaller_pages = [
    Cache(
        "RandomEvictor",
        eviction_strategy=RandomEvictor(),
        size=16
    ),
    Cache(
        "LRU",
        eviction_strategy=LRU(),
        size=16
    ),
    Cache(
        "MQ",
        eviction_strategy=MQ(max_frequency=8, life_time=10, qout_size=8*4),
        size=16
    ),
    Cache(
        "LRU + Markov(3)",
        eviction_strategy=LRU(),
        prefetch_strategy=Markov(3),
        size=16
    ),
    Cache(
        "LRU + NEXT",
        eviction_strategy=LRU(),
        prefetch_strategy=Next(),
        size=16
    ),
    Cache(
        "MQ + NEXT",
        eviction_strategy=MQ(max_frequency=8, life_time=10, qout_size=8*4),
        prefetch_strategy=Next(),
        size=16
    ),
    Cache(
        "MQ + Markov(3)",
        eviction_strategy=MQ(max_frequency=8, life_time=10, qout_size=8*4),
        prefetch_strategy=Markov(3),
        size=16
    ),
]


### Page size - 512 Bytes

In [532]:
test(caches_size_64, test_dirs, 16, page_size=512)


,test_dir,cache,miss_count,trace_length,unique_addresses,page_size,execution_time,miss_rate
0,1,OPT,961,61932,41935,512,None,0.015517
0,1,MQ + Markov(3),51928,61932,41935,512,14.455,0.838468
0,1,MQ + NEXT,51956,61932,41935,512,14.73,0.838920
0,1,LRU + NEXT,51961,61932,41935,512,6.371,0.839001
0,1,LRU + Markov(3),53040,61932,41935,512,7.199,0.856423
0,1,LRU,53757,61932,41935,512,9.004,0.868000
0,1,MQ,53765,61932,41935,512,10.213,0.868130
0,1,RandomEvictor,54045,61932,41935,512,6.805,0.872651
0,11,OPT,209,57867,29936,512,None,0.003612
0,11,MQ + Markov(3),45893,57867,29936,512,15.915,0.793077
